# Student Sampling for Proof of Concept

## Tables used
- student_selection.csv : contains 70 students selected based on their grades (from 100 down to under 40);the idea is to have distributed grades when the final module mark is calculated;
- studentInfo.csv : (Original) Student Info Table which contains demographics
- studentAssessment.csv : (Original) Table which contains student assessment scores
- assesssments.csv : (Original) Table which contains assessment weights

In [1]:
# Import Libraries
import pandas as pd
import numpy as np

In [ ]:
# Read the .csv files from Data directory
studentOriginalDf = pd.read_csv('../Data/studentInfo.csv')
# studentOriginalDf.head()

studentSelectionDf = pd.read_csv('../Data/student_selection.csv')
# studentSelectionDf.head()


In [6]:
# Assuming you already have studentOriginalDf and studentSelectionDf data frames loaded
# Merge the two data frames based on id_student column
subset_df = pd.merge(studentOriginalDf, studentSelectionDf, left_on='id_student', right_on='select_student_id')

# Drop the select_student_id column as it's redundant
subset_df.drop(columns=['select_student_id'], inplace=True)

# Remove records with final_result = 'Withdrawn'
subset_df = subset_df[subset_df['final_result'] != 'Withdrawn']

# Save the subset data frame to a new CSV file
subset_df.to_csv('../Data/subset_students_no_withdrawn.csv', index=False)

print("Subset of studentOriginalDf containing selected students (excluding Withdrawn) has been saved to subset_students_no_withdrawn.csv")


Subset of studentOriginalDf containing selected students (excluding Withdrawn) has been saved to subset_students_no_withdrawn.csv


In [8]:
# Create a new .csv file with unique id_student values and no "Withdrawn" records
# Read the CSV file created above into a pandas DataFrame
subset_df = pd.read_csv('../Data/subset_students_no_withdrawn.csv')

# Remove rows with duplicated id_student values
subset_df.drop_duplicates(subset='id_student', keep='first', inplace=True)

# Save the subset data frame to a new CSV file
subset_df.to_csv('../Data/subset_students_unique.csv', index=False)


In [10]:
# Create a subset .csv file from stundentAssessment file, containing only the assessments of the students in sample unique file
studentAssessment = pd.read_csv('../Data/studentAssessment.csv')
subset_students_unique = pd.read_csv('../Data/subset_students_unique.csv')

# Merge the two data frames based on id_student column
subset_df = pd.merge(studentAssessment, subset_students_unique, left_on='id_student', right_on='id_student')
# subset_df.info()

subset_df.to_csv('../Data/subset_studentAssessment.csv', index=False)

In [12]:
# Load subset_studentAssessment and assessments data frames
subset_studentAssessment = pd.read_csv('../Data/subset_studentAssessment.csv')
assessments = pd.read_csv('../Data/assessments.csv')

# Merge the two data frames based on id_assessment column
merged_df = pd.merge(subset_studentAssessment, assessments[['id_assessment', 'assessment_type', 'date', 'weight']],
                     on='id_assessment', how='left')

# Create the merged DataFrame
merged_df.to_csv('../Data/merged_student_assessment_weights.csv', index=False)


In [ ]:
import numpy as np

# Load 'student' and 'marks' DataFrames
student = pd.read_csv('../Data/subset_students_unique.csv')
marks = pd.read_csv('../Data/merged_student_assessment_weights.csv')

# Group the 'marks' DataFrame by 'id_student'
grouped_marks = marks.groupby('id_student')

# Initialize lists to store final values for each student
final_scores = []
assessment_types = []
date_averages = []

# Define a function to calculate final_score, assessment_type, and date_average
def calculate_values(df):
    # Calculate final_score
    exam_score = df[df['assessment_type'] == 'Exam']['score'].values
    other_scores = df[df['assessment_type'] != 'Exam']['score'].values
    if len(exam_score) > 0:
        final_score = (np.mean(other_scores) + exam_score[0]) / 2
    else:
        final_score = np.mean(other_scores)
    final_scores.append(final_score)
    
    # Calculate assessment_type
    assessment_type_counts = df['assessment_type'].value_counts()
    max_count = max(assessment_type_counts)
    most_common_types = assessment_type_counts[assessment_type_counts == max_count].index.tolist()
    assessment_type = np.random.choice(most_common_types)
    assessment_types.append(assessment_type)
    
    # Calculate date_average
    date_average = np.mean(df[df['assessment_type'] != 'Exam']['date'])
    date_averages.append(date_average)

# Apply the function to each group in grouped_marks
grouped_marks.apply(calculate_values)

# Add new columns to 'student' DataFrame
student['final_score'] = final_scores
student['assessment_type'] = assessment_types
student['date_average'] = date_averages

# Round final_score values to 1 decimal place
student['final_score'] = student['final_score'].round(1)

# Convert date_average values to integers (no decimals)
student['date_average'] = student['date_average'].astype(int)

# Print the updated 'student' DataFrame
print(student)


In [ ]:
# Define a function to map final_score to grading_scale categories
def map_grading_scale(final_score):
    if final_score >= 90:
        return 'A+'
    elif final_score >= 85:
        return 'A'
    elif final_score >= 80:
        return 'A-'
    elif final_score >= 77:
        return 'B+'
    elif final_score >= 73:
        return 'B'
    elif final_score >= 70:
        return 'B-'
    elif final_score >= 67:
        return 'C+'
    elif final_score >= 63:
        return 'C'
    elif final_score >= 60:
        return 'C-'
    elif final_score >= 50:
        return 'D'
    else:
        return 'F'

# Apply the function to create the grading_scale column
student['grading_scale'] = student['final_score'].apply(map_grading_scale)

# Print the updated 'student' DataFrame
print(student)


In [ ]:
# Load and 'courses' DataFrame
courses = pd.read_csv('../Data/courses.csv')

# Merge the 'student' and 'courses' DataFrames based on code_module and code_presentation
merged_student_course = pd.merge(student, courses, on=['code_module', 'code_presentation'], how='left')

# Print the updated 'student' DataFrame with the 'module_presentation_length' column added
print(merged_student_course)


In [ ]:
# Select the desired columns
finalStudent = merged_student_course[['gender', 'region', 'highest_education', 'imd_band', 'age_band',
                               'num_of_prev_attempts', 'studied_credits', 'disability',
                               'assessment_type', 'date_average', 'module_presentation_length',
                               'grading_scale']]

# Write the finalStudent DataFrame to a CSV file
finalStudent.to_csv('final_student.csv', index=False)

# Print a message to confirm the CSV file has been created
print("final_student.csv has been created successfully.")
